In [1]:
import os
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join

In [2]:
directory = r'C:\Users\shirahad\Documents\research\electrophysiology\analyze_electrophysiology_data\psth\files'

# PAY ATTENTION TO 7 OR 10 INTENSITIES

In [3]:
intensities_file_path = r'C:\Users\shirahad\Documents\research\electrophysiology\analyze_electrophysiology_data\pl2kilosort\7_intensities.csv'
intensities_df = pd.read_csv(intensities_file_path, header=0, index_col=0)
intensities_df

,ND,intensity
0,7,9.354108
1,6,10.354108
2,5,11.354108
3,4,12.354108
4,3,13.354108
5,2,14.354108
6,1,15.354108


In [13]:
clustering_file_path = r'D:\new_mapping_results\clustering\PHb\clustering_data.csv'
clustering_df = pd.read_csv(clustering_file_path, header=0, index_col=0)

In [14]:
clusters_groups = clustering_df[['cluster', 'prefix']].groupby('prefix')

In [15]:
files = [f for f in listdir(directory) if isfile(join(directory, f))]

In [17]:
working_directory = r'C:\Users\shirahad\Documents\research\electrophysiology\analyze_electrophysiology_data\psth'
idx = pd.IndexSlice

for f in files:
    group = [group for (key, group) in clusters_groups if f.startswith(key)][0]
    response_df = pd.read_csv(os.path.join(directory, f), header=[0, 1, 2], index_col=0)
    nds = response_df.columns.get_level_values('nd').unique()
    cluster_ids = response_df.columns.get_level_values('id').unique()
    mean_df = pd.DataFrame(columns=[cluster_ids.sort_values(), group['cluster']], index=nds)
    mean_df = mean_df.apply(lambda x: response_df.loc[:, idx[x.name, :, 'mean']].T.set_index(intensities_df['intensity'], drop=True)[0])
    mean_df.to_csv(r'{}\files\{}_mean.csv'.format(working_directory, f[: -4]), sep=',', encoding='utf-8')